# Persistent homology analysis of CA1 Hippocampus data

## Set up environment paths

In [ ]:
import setup
setup.main()

%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lovely_numpy import lo
from evaluate import compute_persistence_diagrams
from viz import plot_persistence_diagrams
import neuralgeom.datasets.experimental as experimental
import seaborn as sns

## Load neural activity

In [ ]:
expt_id = "34"
timestep_microsec = int(2e5)
vel_threshold = 5

neural_activity, labels = experimental.load_neural_activity(expt_id=expt_id, vel_threshold= vel_threshold, timestep_microsec=timestep_microsec)

times_in_seconds = labels["times"]*1e-6
angles = labels["angles"]

print(f"There are {neural_activity.shape[1]} neurons binned over {neural_activity.shape[0]} timesteps")

In [ ]:
from sklearn.feature_selection import mutual_info_regression
angles_radians = angles*np.pi/180

X = np.cos(angles_radians)
Y = np.sin(angles_radians)

spatial_mutual_info_X = mutual_info_regression(neural_activity,X)
spatial_mutual_info_Y = mutual_info_regression(neural_activity,Y)
spatial_mutual_info_XY = spatial_mutual_info_X + spatial_mutual_info_Y


mi_threshold = 0.15
mi_thresh_neural_activity = neural_activity[:,spatial_mutual_info_XY>mi_threshold]

In [ ]:
diagrams = compute_persistence_diagrams(mi_thresh_neural_activity, maxdim=2, n_threads=-1)

In [ ]:
plot_persistence_diagrams(diagrams)

In [ ]:
import pandas as pd

# Create empty list to store dataframes
dfs = []

# Iterate over each diagram
for i, diagram in enumerate(diagrams):
    # Convert numpy array to pandas DataFrame
    df = pd.DataFrame(diagram, columns=['Birth', 'Death'])
    # Add dimensionality as a column
    df['Dimension'] = i
    # Append DataFrame to the list
    dfs.append(df)

# Concatenate all dataframes
result_df = pd.concat(dfs, ignore_index=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_persistence_diagrams_seaborn(result_df):
    # Determine the max limit for x and y axes
    max_limit = max(result_df['Death'].max(), result_df['Birth'].max())

    # Create a grid of scatterplots, one for each dimension
    grid = sns.FacetGrid(result_df, col="Dimension", hue="Dimension", 
                         height=5, xlim=(0, max_limit), ylim=(0, max_limit))

    # Map scatterplot onto the grid
    grid.map(sns.scatterplot, 'Birth', 'Death', s=10)

    # Plot 45-degree line
    grid.map(plt.plot, 'Birth', 'Death', ls="--", c='gray')

    # Set grid aesthetics
    grid.add_legend()
    grid.set_axis_labels("Birth", "Death")
    grid.fig.suptitle("Persistence Diagram")

    plt.show()

plot_persistence_diagrams_seaborn(result_df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_persistence_diagrams_seaborn(result_df):
    # Make a copy of the dataframe to avoid changing the original data
    plot_df = result_df.copy()

    # Find the maximum finite value from the Birth and Death columns
    max_finite_value = plot_df.replace([np.inf, -np.inf], np.nan).max().max()

    # Replace infinities with a slightly larger value than the maximum finite value
    plot_df.replace([np.inf, -np.inf], max_finite_value * 1.1, inplace=True)

    # Determine the max limit for x and y axes
    max_limit = max(plot_df['Death'].max(), plot_df['Birth'].max())

    # Create a grid of scatterplots, one for each dimension
    grid = sns.FacetGrid(plot_df, col="Dimension", hue="Dimension", 
                         height=5, xlim=(0, max_limit), ylim=(0, max_limit))

    # Map scatterplot and kdeplot onto the grid
    grid.map(sns.scatterplot, 'Birth', 'Death', s=10)
    

    # Define a function to draw filled kde plot
    def kde_fill(data,**kwargs):
        sns.kdeplot(data, x = "Birth", y = "Death",fill=True, thresh=0, levels=100, cmap='Reds', alpha=0.5, **kwargs)

    # Map kde_fill function onto the grid
    grid.map_dataframe(kde_fill)

    def plot_45_degree_line(**kwargs):
        plt.plot([0, max_limit], [0, max_limit], ls="--", c='gray')

    # Map function to plot 45-degree line onto the grid
    grid.map(plot_45_degree_line)

    # Set grid aesthetics
    grid.add_legend()
    grid.set_axis_labels("Birth", "Death")
    grid.fig.suptitle("Persistence Diagram")

    plt.show()

plot_persistence_diagrams_seaborn(result_df)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_persistence_diagrams_seaborn(result_df):
    # Make a copy of the dataframe to avoid changing the original data
    plot_df = result_df.copy()

    # Find the maximum finite value from the Birth and Death columns
    max_finite_value = plot_df.replace([np.inf, -np.inf], np.nan).max().max()

    # Replace infinities with a slightly larger value than the maximum finite value
    plot_df.replace([np.inf, -np.inf], max_finite_value * 1.1, inplace=True)

    # Determine the max limit for x and y axes
    max_limit = max(plot_df['Death'].max(), plot_df['Birth'].max())

    # Create a grid of scatterplots, one for each dimension
    grid = sns.FacetGrid(plot_df, col="Dimension", hue="Dimension", 
                         height=5, xlim=(0, max_limit), ylim=(0, max_limit))

    # Map scatterplot onto the grid
    grid.map(sns.scatterplot, 'Birth', 'Death', s=10)

    # Plot 45-degree line
    grid.map(plt.plot, 'Birth', 'Death', ls="--", c='gray')

    # Set grid aesthetics
    grid.add_legend()
    grid.set_axis_labels("Birth", "Death")
    grid.fig.suptitle("Persistence Diagram")

    plt.show()

plot_persistence_diagrams_seaborn(result_df)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))


sns.kdeplot(result_df[result_df["Dimension"] == 0],x="Birth",y="Death",hue="Dimension",fill=True, ax=ax)

ax.set_xlim(0, 25)
ax.set_ylim(0, 25);

ax.plot([0, 60], [0, 60], color='r', linestyle='--')

In [ ]:
sns.displot(result_df,x="Birth",y="Death",hue="Dimension",kind="kde",fill=True)